In [1]:
import h5py
import glob
import numpy as np
import random
from settings import *

In [5]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
# pick which random coords to display:
f = h5py.File('/data/Schema/intact/fsaverage6_adj.h5', 'r')
dispcoordsLH = f['lhinflatedcoords'][:]
randcoords = random.sample(range(len(dispcoordsLH[1])), 5000)
dispcoordsLH = dispcoordsLH[:,randcoords]
dispcoordsRH = f['rhinflatedcoords'][:]
dispcoordsRH = dispcoordsRH[:,randcoords]

fig1 = plt.figure()

f = h5py.File(h5path+'ISCtest3.h5', 'r')
#f = h5py.File(
#    '/data/HBN/test/fmriprep_output/fmriprep/PythonData/ISC.h5', 'r')

for idx,task in enumerate(['DM']):#,'TP']):
    ISC = np.nan_to_num(f[task]['ISC'][:])
    # For SUMA visualization:
    np.savetxt(h5path+task+'ISC.txt',ISC)
    np.savetxt(h5path+task+'ISC_LH.txt',ISC[:len(ISC)//2])
    np.savetxt(h5path+task+'ISC_RH.txt',ISC[len(ISC)//2:])
    dispISCLH = ISC[:len(ISC)//2][randcoords] # For LH
    dispISCRH = ISC[len(ISC)//2:][randcoords] # For RH
    # plot ISC values across cortex
    axrh = fig1.add_subplot(2,2,idx+1, projection='3d')
    x,y,z = dispcoordsRH
    brainrh = axrh.scatter3D(x,y,z,c=dispISCRH, cmap='bwr')
    fig1.colorbar(brainrh)
    axrh.set_title(task + ' RH')
    
    axlh = fig1.add_subplot(2,2,idx+3, projection='3d')
    x,y,z = dispcoordsLH
    brainlh = axlh.scatter3D(x,y,z,c=dispISCLH, cmap='bwr');
    fig1.colorbar(brainlh)
    axlh.set_title(task + ' LH')

FigureCanvasNbAgg()

In [8]:
fig2 = plt.figure()

Ages = []
Sexes = []
for s in f['subord'][:]:
    fs = h5py.File(s,'r')
    Ages.append(fs['Pheno']['Age'].value)
    Sexes.append(fs['Pheno']['Sex'].value)
Agesr = np.round(Ages)
Agesu = list(set(Agesr))

for idx,task in enumerate(['DM']):#,'TP']):    
    ISC_persubj = np.mean(np.nan_to_num(f[task]['ISC_persubj'][:])>0.1,axis=0)
    # ISC_persubj = np.nanmean(f[task]['ISC_persubj'][:],axis=0)
    ISCam = []
    ISCas = []
    for a in Agesu:
        idxs = list(np.where(Agesr==a))[0]
        ISCam.append(np.mean(ISC_persubj[idxs]))
        ISCas.append(np.std(ISC_persubj[idxs]))
    ISCsm = []
    ISCss = []
    for s in np.int64([0,1]):
        idxs = list(np.where(Sexes==s))[0]
        ISCsm.append(np.mean(ISC_persubj[idxs]))
        ISCss.append(np.std(ISC_persubj[idxs])) 
    # plot per-subject ISC values by age
    ax = fig2.add_subplot(2,2,idx+1)
    ax.errorbar(Agesu, ISCam, ISCas, marker='o', linestyle='None')
    plt.ylabel('ISC')
    plt.xlabel('Age')
    ax.set_title(task + ' ISC by age')
    # plot per-subject ISC values by sex
    ax = fig2.add_subplot(2,2,idx+3)
    ax.errorbar([0,1], ISCsm, ISCss, linestyle='None', marker='^')
    ax.xaxis.set_ticks([0, 1])
    ax.xaxis.set_ticklabels(('Male','Female'))
    ax.set_title(task + ' ISC by sex')
    plt.ylabel('ISC')
fig2.tight_layout()

FigureCanvasNbAgg()

/home/scohen/.conda/envs/scohen_brainiak_env_0.8/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/scohen/.conda/envs/scohen_brainiak_env_0.8/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/scohen/.conda/envs/scohen_brainiak_env_0.8/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/scohen/.conda/envs/scohen_brainiak_env_0.8/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/scohen/.conda/envs/scohen_brainiak_env_0.8/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# plot matrix of per-subject ISCs



In [6]:
%matplotlib widget
fig = plt.figure()
#ax = plt.axes(projection='3d')
ax = fig.gca(projection='3d')

x,y,z = dispcoordsRH
brain = ax.scatter3D(x,y,z,c=dispISCRH, cmap='bwr');
plt.colorbar(brain)

FigureCanvasNbAgg()